# NPI-5: NPI is 10 digits

Description: check if all the NPIS are exactly 10 digits. Test will pass if it is true.

Starting Author: Amy Jin (amy@careset.com)

Date: April 30th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, test will print out all distinct failed NPIs.


def npi_5(db_name, table_name, col_name):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
            SELECT *
            FROM {db}.{t1}
            WHERE length({col1}) <> 10
            LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        
        if not len(rows):
            print ("Npi_5 test result: PASS")
        else:
            print ("Npi_5 test result: FAIL" + '\n')
            print ("The following {} fail the test:".format(col_name) + '\n')
            
            # MySQL query to get failed NPIs
            query = ('''
                SELECT DISTINCT {col1}
                FROM {db}.{t1}
                WHERE length({col1}) <> 10;
            '''.format(db = db_name, t1 = table_name, col1 = col_name))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()

## Test Example

* SELECT length(npi) AS num_digit, npi
* FROM drg_npi.drg_npi_2014_2015
* WHERE length(npi) <> 10;

In [3]:
npi_5('_amy', 'test_data_good', 'npi_dest')

Npi_5 test result: PASS


In [3]:
npi_5('client_celgene',' HCP_Geographic_distribution_of_MM_Patients', '`Physician NPI Number`')

Npi_5 test result: PASS


In [4]:
npi_5('_amy', 'test_data_bad1', 'npi')

Npi_5 test result: FAIL

The following npi fail the test:

-3890512889, 

317195182, 

hello, 

744882010, 

-1034967478, 

-2977027415, 

-7808887280, 

, 

-1377764121, 

246798161, 

-6740599147, 

-1680547904, 

-1940208221, 

-3733149805, 

-4788451357, 

-7240162248, 

-4946354333, 

-2965820727, 

-6306254008, 

-3674108797, 

-1562555165, 

-5310822741, 

-1941542794, 

-6736649911, 

-4429541401, 

507389324, 

-7276831325, 

-6374967444, 

-2794293886, 

-5778586681, 

-4851641689, 

-6595396093, 

-3971089833, 

-7918653799, 

-7512599943, 

-1510458706, 

-3399600624, 

-2054924448, 

-4813093985, 

-3768680897, 

-6425644737, 

-7577750907, 

-6719102102, 

678168575, 

-6008090112, 

-7109095853, 

-3315192347, 

-6929077466, 

-5468995512, 

-3703480463, 

-1962309126, 

-6181925666, 

-6323672995, 

-2224326439, 

-6908799060, 

-7167918298, 

-3754510236, 

-1001306713, 

-6947459044, 

-2144945416, 

-6254492840, 

-5064661882, 

-7885987898, 

-1000469978, 

-7026180